# 从数字格式化到函数组合

假设需实现一数字格式化函数
- 格式上支持百分比，万、十万等后缀；
- 精度上支持自定义小数位数；
- 四舍五入、向上向下取整；
- 千分位分隔；
- 自定义前后缀。

具体见ts定义。

## 接口定义
```ts
export type FormatRules = Partial<{
  format: string
  precision: number
  carry: 'floor' | 'ceil' | 'round'
  thousandsSeparator: boolean
}>

type FormatEnum =
  | 'percent'
  | 'wan' // 万
  | 'baiwan'  // 百万
  | 'qianwan' // 千万
  | 'yi' // 亿

export function formatNumber(value: number, opts: FormatRules);
```

## format suffix

格式化后会加上合适的后缀。

In [58]:
function getFormatSuffix(format) {
  const map = {
    percent: '%',
    wan: '万',
    baiwan: '百万',
    qianwan: '千万',
    yi: '亿',
  }
  return map[format]
}

getFormatSuffix('percent')

'%'

In [4]:
getFormatSuffix('yi') // 不好意思，用了中文拼音，有点邪恶。

'亿'

## auto

当 format未指定时，需“推断”出合适的 format, 当然只对上述几个单位有效。

In [5]:
function detectFormat(value) {
  const list = [
    [1_0000_0000, 'yi'],
    [1000_0000, 'qianwan'],
    [100_0000, 'baiwan'],
    [1_0000, 'wan'],
  ]
  const item = list.find(pair => value >= pair[0])
  return item ? item[1] : undefined
}

In [6]:
detectFormat(123456789)

'yi'

In [11]:
detectFormat(98765)

'wan'

In [10]:
detectFormat(123)

## 千分位

我觉得正则表达式的版本会比循环版本好。

In [18]:
function formatWithThousandsSeparator(num/*:string*/) {
  const [integerPart, decimalPart] = num.split('.')
  const formattedInteger = integerPart.replace(/\B(?=(\d{3})+(?!\d))/g, ',')
  return decimalPart ? `${formattedInteger}.${decimalPart}` : formattedInteger
}

In [19]:
formatWithThousandsSeparator('123456789')

'123,456,789'

这个正则表达式是GPT帮我写的，我对正则表达式的环视语法不大熟悉：（。

## 舍入
四舍五入和上下取整直接使用Number提供的函数，
注意的是，这个方法只对整数操作， 而我们要求中的舍入是针对小数位数的。

In [22]:
function getCarryValue(value, type) {
  if (type === 'floor') {
    return Math.floor(value)
  }
  if (type === 'ceil') {
    return Math.ceil(value)
  }
  return Math.round(value)
}

## 格式化缩放

准备了一些工具方法后，进入正题，首先根据格式要求对数据缩放。

In [30]:
function getScaledValue(value, format) {
  const shift = getFormatShift(format)
  const next = shift !== 0 ? value * 10 ** shift : value
  return next
}

function getFormatShift(format) {
  const map = {
    percent: 2,
    // 万
    wan: -4,
    // 百万
    baiwan: -6,
    // 千万
    qianwan: -7,
    // 亿
    yi: -8,
  }
  return format ? map[format] ?? 0 : 0
}

In [59]:
getScaledValue(0.785, 'percent')

78.5

In [60]:
getScaledValue(12345, 'wan')

1.2345

## 保留有效小数位数

用到上述定义的舍入函数： `getCarryValue()`

In [34]:
function getPrecisionValue(value, precision, type) {
  const scale = 10 ** precision
  const next = getCarryValue(value * scale, type)
  return next / scale
}


In [36]:
getPrecisionValue(123.123, 2, 'round')

123.12

In [66]:
getPrecisionValue(123.125, 2, 'round')

123.13

## 浮点数精度

In [67]:
0.1 + 0.2

0.30000000000000004

由于JS对浮点数计算的精度问题，结合当前的应用场景，使用round来避免以上问题。

In [72]:
function round(value) {
  const scale = 10 ** 8
  return Math.round(value * scale) / scale
}

In [46]:
round(0.1 + 0.2)

0.3

## 整合起来

原料都准备好了，现在是时候整合起来了。

In [77]:
function formatNumber(value, opts = {}) {
  const format = opts.format === 'auto' ? detectFormat(value) : opts.format
  const precision = opts.precision ?? -1
  const next = getScaledValue(value, format)
  const result = precision >= 0 ? getPrecisionValue(next, precision, opts.carry) : next
  const str = toString(result, precision, format, opts)
  return str
}

最后的 `toString` 用于格式化

```ts
export type FormatRules = Partial<{
  format: string
  precision: number
  carry: 'floor' | 'ceil' | 'round'
  thousandsSeparator: boolean
}>
```

In [114]:
function toString(num, precision, format, opts) {
  const value = precision >= 0 ? num.toFixed(precision) : `${round(num)}` // <-- 注意第二个分支
  const next = opts.thousandsSeparator ? formatWithThousandsSeparator(value) : value
  const formatSuffix = format ? getFormatSuffix(format) : ''
  return `${next}${formatSuffix}`
}

In [79]:
formatNumber(1234512345.6789, { format: 'wan', precision: 2, carry: 'round', thousandsSeparator: true })

'123,451.23万'

## 默认精度

当包含精度参数(precision）时，toFixed会在内部帮我们处理好浮点数的精度问题；  
否则我们需要自己这里， 这是上面 `${round(num)}` 的作用。

In [108]:
0.2+0.7

0.8999999999999999

In [109]:
(0.2+0.7) * 234

210.59999999999997

In [110]:
formatNumber((0.2+0.7) * 234)

'210.6'